# NSSP project 1
### preprocessing and GLM analysis

In this notebook, we aims to study the neural processing of emotionally provocative auditory stimuli.

# Setup the notebook

### Import, setup FSL

In [1]:
%gui wx
import sys
import os
import os.path as op

#utility functions from previous labs
import utils 

#to download the dataset from openneuro
import subprocess
import openneuro

#############################
# Loading fsl and freesurfer within Neurodesk
# You can find the list of available other modules by clicking on the "Softwares" tab on the left
#############################
import lmod
await lmod.purge(force=True)
await lmod.load('fsl/6.0.7.4')
await lmod.load('freesurfer/7.4.1')
await lmod.list()

###################
# Load all relevant libraries for the lab
##################
import fsl.wrappers
from fsl.wrappers import fslmaths

# FSL function wrappers which we will call from python directly
from fsl.wrappers import fast, bet
from fsl.wrappers.misc import fslroi
from fsl.wrappers import flirt
from fsl.wrappers import mcflirt

from mne.datasets import sample
from mne_bids import BIDSPath, read_raw_bids, print_dir_tree, make_report

#concatenation
import mne_nirs
import nilearn
from nilearn.image import concat_imgs, mean_img
import nibabel as nib

import ants

# General purpose imports to handle paths, files etc
import glob
import pandas as pd
import numpy as np
import json
import subprocess

import matplotlib.pyplot as plt

#GLM
import nibabel as nib
from nilearn.glm import threshold_stats_img
from nilearn.plotting import plot_stat_map,plot_design_matrix,plot_contrast_matrix
from nilearn.glm.first_level import make_first_level_design_matrix, FirstLevelModel

Gtk-Message: 08:20:23.868: Failed to load module "canberra-gtk-module"


In [3]:
#We define the parameters RERUN to rerun all the transform, if false it will use the saved files.
RERUN = False

# Pre-processing